<a href="https://colab.research.google.com/github/nd0010/Assignment/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install --quiet cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.9 MB/s eta 0:00:00


In [3]:
!pip install --quiet qdrant-client python-dotenv langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
pip install --quiet wikipedia

  Preparing metadata (setup.py) ... done


In [5]:
import cohere
import pandas as pd
import numpy as np
import altair as alt
import textwrap as tr
import random
import dotenv
import os

dotenv.load_dotenv(".env")

True

In [6]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain.retrievers import WikipediaRetriever

In [7]:
retriever = WikipediaRetriever()

docs = retriever.get_relevant_documents("Pizza")

pizza_doc = docs[0].page_content

#print (pizza_doc)

pizza_doc_split = pizza_doc.split("\n\n",3)
print(pizza_doc_split)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

#pizza_text_split = text_splitter.create_documents(pizza_doc)
#print (pizza_text_split[0])
#print (pizza_text_split[1])

pizza_text_split = text_splitter.split_text(pizza_doc)
print (pizza_text_split)


["Pizza ( PEET-sə, Italian: [ˈpittsa]; Neapolitan: [ˈpittsə]) is a dish of Italian origin consisting of a usually round, flat base of leavened wheat-based dough topped with tomatoes, cheese, and often various other ingredients (such as anchovies, mushrooms, onions, olives, vegetables, meat, etc.), which is then baked at a high temperature, traditionally in a wood-fired oven.The term pizza was first recorded in the year 997, in a Latin manuscript from the southern Italian town of Gaeta, in Lazio, on the border with Campania. Raffaele Esposito is often credited for creating modern pizza in Naples. In 2009, Neapolitan pizza was registered with the European Union as a traditional speciality guaranteed dish. In 2017, the art of making Neapolitan pizza was added to UNESCO's list of intangible cultural heritage.Pizza and its variants are among the most popular foods in the world. Pizza is sold at a variety of restaurants, including pizzerias (pizza specialty restaurants), Mediterranean restau

In [8]:
model = "multilingual-22-12"
embeddings = CohereEmbeddings (model = "multilingual-22-12")

db = Qdrant.from_texts(pizza_text_split, embeddings, location =":memory:", collection_name="my_documents", distance_func="Dot")

In [9]:
queries = ["What is Pizza",
           "What are the ingredients in a Pizza",
           "Does Pizza come in square sizes",
           "When was Pizza invented"]

In [10]:
answers = []

for query in queries :
  docs = db.similarity_search(query)
  answers.append(docs[0].page_content)

answers

['Pizza ( PEET-sə, Italian: [ˈpittsa]; Neapolitan: [ˈpittsə]) is a dish of Italian origin consisting',
 'Pizza ( PEET-sə, Italian: [ˈpittsa]; Neapolitan: [ˈpittsə]) is a dish of Italian origin consisting',
 'Pizza is sold at a variety of restaurants, including pizzerias (pizza specialty restaurants),',
 'a high temperature, traditionally in a wood-fired oven.The term pizza was first recorded in the']